# Validating measures

In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import time

### Data Directory

In [2]:
INPUT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/processed/'

### Reading predicted file

In [3]:
ip_file = INPUT_DIR + 'fd004_dbscan_results_1450_0.001.csv'
df_pred = pd.read_csv(ip_file, sep=',')

In [4]:
df_pred.head()

,setting1,setting2,meas02,meas03,meas04,meas06,meas07,meas08,meas09,meas11,meas12,meas13,meas14,meas15,meas17,meas20,meas21,unit,time_cycles,labels
0,42.0049,0.8400,549.68,1343.43,1112.93,5.70,137.36,2211.86,8311.32,41.69,129.78,2387.99,8074.83,9.3335,330,10.62,6.3670,1,1,0
1,20.0020,0.7002,606.07,1477.61,1237.50,13.61,332.10,2323.66,8713.60,43.94,312.59,2387.73,8046.13,9.1913,361,24.37,14.6552,1,2,1
2,42.0038,0.8409,548.95,1343.12,1117.05,5.69,138.18,2211.92,8306.69,41.66,129.62,2387.97,8066.62,9.4007,329,10.48,6.4213,1,3,0
3,42.0000,0.8400,548.70,1341.24,1118.03,5.70,137.98,2211.88,8312.35,41.68,129.80,2388.02,8076.05,9.3369,328,10.54,6.4176,1,4,0
4,25.0063,0.6207,536.10,1255.23,1033.59,9.00,174.82,1915.22,7994.94,36.48,164.11,2028.08,7865.80,10.8366,305,14.03,8.6754,1,5,2


In [5]:
# Make everything in labels column into 0 or -1
df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)

### Generating Ground Truth with heuristics

In [6]:
timecycles_for_failure = 10

In [7]:
df_pred['ground_truth'] = 0

In [8]:
df_pred.head()

,setting1,setting2,meas02,meas03,meas04,meas06,meas07,meas08,meas09,meas11,...,meas13,meas14,meas15,meas17,meas20,meas21,unit,time_cycles,labels,ground_truth
0,42.0049,0.8400,549.68,1343.43,1112.93,5.70,137.36,2211.86,8311.32,41.69,...,2387.99,8074.83,9.3335,330,10.62,6.3670,1,1,0,0
1,20.0020,0.7002,606.07,1477.61,1237.50,13.61,332.10,2323.66,8713.60,43.94,...,2387.73,8046.13,9.1913,361,24.37,14.6552,1,2,0,0
2,42.0038,0.8409,548.95,1343.12,1117.05,5.69,138.18,2211.92,8306.69,41.66,...,2387.97,8066.62,9.4007,329,10.48,6.4213,1,3,0,0
3,42.0000,0.8400,548.70,1341.24,1118.03,5.70,137.98,2211.88,8312.35,41.68,...,2388.02,8076.05,9.3369,328,10.54,6.4176,1,4,0,0
4,25.0063,0.6207,536.10,1255.23,1033.59,9.00,174.82,1915.22,7994.94,36.48,...,2028.08,7865.80,10.8366,305,14.03,8.6754,1,5,0,0


In [9]:
units = np.unique(df_pred['unit'])

In [10]:
df_pred.loc[df_pred['unit'] == -1]

,setting1,setting2,meas02,meas03,meas04,meas06,meas07,meas08,meas09,meas11,...,meas13,meas14,meas15,meas17,meas20,meas21,unit,time_cycles,labels,ground_truth


In [11]:
# Assign outliers for the last 'n' time cycles for each of the unit

units = np.unique(df_pred['unit'])
# print('Number of unique units:', len(units))
for current_unit in units:
    df_sub = df_pred.loc[df_pred['unit'] == current_unit]
    max_timecycle = max(df_sub['time_cycles'])    
    outlier_time_threshold = max_timecycle - timecycles_for_failure
    
    # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)
    
    df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1

/Users/talat/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
print(df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].loc[df_pred['unit'] == 1])

     unit  time_cycles  labels  ground_truth
0       1            1       0             0
1       1            2       0             0
2       1            3       0             0
3       1            4       0             0
4       1            5       0             0
5       1            6       0             0
6       1            7       0             0
7       1            8       0             0
8       1            9       0             0
9       1           10       0             0
10      1           11       0             0
11      1           12       0             0
12      1           13       0             0
13      1           14       0             0
14      1           15       0             0
15      1           16       0             0
16      1           17       0             0
17      1           18       0             0
18      1           19       0             0
19      1           20       0             0
20      1           21       0             0
21      1 

In [13]:
print('Number of outliers in the predicted values', len(df_pred.loc[df_pred['labels'] == -1]))
print('Number of outliers in the ground truth', len(df_pred.loc[df_pred['ground_truth'] == -1]))

Number of outliers in the predicted values 1009
Number of outliers in the ground truth 2490


In [14]:
# Copy into a new dataframe with only the required columns
df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()

In [15]:
# tp = 0
# fp = 1
# fn = 2
# tn = 3

def gen_vals(ground_truth, pred_label, pos_label, neg_label):
    if (ground_truth == pos_label) & (pred_label == pos_label):
        return 0
    elif (ground_truth == neg_label) & (pred_label == pos_label):
        return 1
    elif (ground_truth == pos_label) & (pred_label == neg_label):
        return 2
    elif (ground_truth == neg_label) & (pred_label == neg_label):
        return 3

### Find out the f-measure

In [16]:
pos_label = -1
neg_label = 0

df_scoring['pos'] = -1

df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)

In [17]:
from collections import Counter
counter = Counter(df_scoring['pos'])

print(counter)

accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
print('Accuracy = ', accuracy)

precision = counter[0] / (counter[0] + counter[1])
print('Precision = ', precision)

recall = counter[0] / (counter[0] + counter[2])
print('Recall = ', recall)

f1_score = 2 * (precision * recall) / (precision + recall)
print('F1 score: ', f1_score)

Counter({3: 58465, 2: 1775, 0: 715, 1: 294})
Accuracy =  0.9662198566507209
Precision =  0.7086223984142715
Recall =  0.28714859437751006
F1 score:  0.4086881966276079


# Making it into a function

In [18]:
def find_eval_metrics(df_pred, timecycles_for_failure):

    # Make everything in labels column into 0 or -1
    df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)
    
    df_pred['ground_truth'] = 0
    
    # Assign outliers for the last 'n' time cycles for each of the unit
    units = np.unique(df_pred['unit'])
    # print('Number of unique units:', len(units))
    for current_unit in units:
        df_sub = df_pred.loc[df_pred['unit'] == current_unit]
        max_timecycle = max(df_sub['time_cycles'])    
        outlier_time_threshold = max_timecycle - timecycles_for_failure

        # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)

        df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1
        
    # Copy into a new dataframe with only the required columns
    df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()
    
    pos_label = -1
    neg_label = 0

    df_scoring['pos'] = -1
    df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)
    
    counter = Counter(df_scoring['pos'])

    accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
    # print('Accuracy = ', accuracy)
    precision = counter[0] / (counter[0] + counter[1])
    # print('Precision = ', precision)
    recall = counter[0] / (counter[0] + counter[2])
    # print('Recall = ', recall)
    f1_score = 2 * (precision * recall) / (precision + recall)
    # print('F1 score: ', f1_score)
    
    return accuracy, precision, recall, f1_score

In [19]:
import warnings
warnings.filterwarnings("ignore")

max_timecycles_for_failure = 40

print('| timecycles_for_failure|', 'accuracy', '|', 'precision', '|', 'recall', '|', 'f1_score', '|')
print('|--|--|--|--|--|')

for i in range(1, max_timecycles_for_failure+1):
    accuracy, precision, recall, f1_score = find_eval_metrics(df_pred, i)
    print('|', i, '|', accuracy, '|', precision, '|', recall, '|', f1_score, '|')

| timecycles_for_failure| accuracy | precision | recall | f1_score |
|--|--|--|--|--|
| 1 | 0.9828242093748469 | 0.10208126858275521 | 0.41365461847389556 | 0.16375198728139906 |
| 2 | 0.9824486930398864 | 0.21407333994053518 | 0.43373493975903615 | 0.28666224286662245 |
| 3 | 0.981191529657627 | 0.2993062438057483 | 0.4042838018741633 | 0.3439635535307517 |
| 4 | 0.980195595030123 | 0.3924677898909812 | 0.39759036144578314 | 0.3950124688279302 |
| 5 | 0.9782527061666313 | 0.4568880079286422 | 0.37028112449799194 | 0.4090505767524401 |
| 6 | 0.9761792029257621 | 0.5173439048562933 | 0.3493975903614458 | 0.4170994806232521 |
| 7 | 0.9743995820339924 | 0.5867195242814668 | 0.33964429145152036 | 0.43023255813953487 |
| 8 | 0.9719342356609904 | 0.6352824578790882 | 0.3217871485943775 | 0.4271909363545485 |
| 9 | 0.9691750069388888 | 0.6749256689791873 | 0.3038821954484605 | 0.41907692307692307 |
| 10 | 0.9662198566507209 | 0.7086223984142715 | 0.28714859437751006 | 0.4086881966276079 |
| 1